In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

import loss
import dotenv
import torch
import matplotlib.pyplot as plt
import mlflow
import os
from torch.utils.data import DataLoader
from intra import IntrA
from functools import partial
from augmentation import apply_augmentations
from models.pointnet2_ssg import PointNet2ClassificationSSG
from models.pointmlp import pointMLP
from models.curvenet import CurveNet
import torch.nn.functional as F
import numpy as np

from utils import train_model, train_kfold_intra, eval_model_classification

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

dotenv.load_dotenv()  # load the MLflow http authentication parameters
mlflow.set_tracking_uri(os.environ.get("MLFLOW_TRACKING_URI"))

dev = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)


# ensure reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print(dev)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'pointnet2_ops'

In [4]:
"""PointMLP full training setup."""
if True:
    trn = IntrA(
        "/data",
        npoints=1024,
        exclude_seg=True,
        norm=False,
        dataset="classification",
        fold=1,
        kfold_splits="./file_splits/cls",
        test=False,
    )

    tst = IntrA(
        "/data",
        npoints=1024,
        exclude_seg=True,
        norm=False,
        dataset="classification",
        fold=1,
        kfold_splits="./file_splits/cls",
        test=True,
    )
 
    train_dl = DataLoader(trn, batch_size=64, num_workers=8, drop_last=True, shuffle=True)
    test_dl = DataLoader(tst, batch_size=8, num_workers=2, drop_last=False)

    model = pointMLP(num_classes=2)

    train_model(
        model,
        train_dl,
        test_dl,
        loss.smooth_cross_entropy_loss(),
        partial(apply_augmentations),
        model_name="PointMLP",
        epochs=300,
        checkpoint_epoch=150,
        trans_loss=False,
        opt="sgd",
        sched="cosine",
        lr=0.025,
        min_lr=0.0005
    )

In [ ]:
"""CurveNet full training setup."""
if True:
    trn = IntrA(
        "/data",
        npoints=1024,
        exclude_seg=True,
        norm=False,
        dataset="classification",
        fold=1,
        kfold_splits="./file_splits/cls",
        test=False,
    )

    tst = IntrA(
        "/data",
        npoints=1024,
        exclude_seg=True,
        norm=False,
        dataset="classification",
        fold=1,
        kfold_splits="./file_splits/cls",
        test=True,
    )
 
    train_dl = DataLoader(trn, batch_size=64, num_workers=8, drop_last=True, shuffle=True)
    test_dl = DataLoader(tst, batch_size=8, num_workers=2, drop_last=False)

    model = CurveNet(num_classes=2)

    train_model(
        model,
        train_dl,
        test_dl,
        loss.smooth_cross_entropy_loss(),
        partial(apply_augmentations, rotation=False, translation=False),
        model_name="CurveNet",
        epochs=300,
        checkpoint_epoch=150,
        trans_loss=False,
        opt="sgd",
        sched="cosine",
        lr=0.05,
        min_lr=0.0001
    )

In [ ]:
# dataset = IntrA(
#         "/data",
#         npoints=1024,
#         exclude_seg=True,
#         norm=False,
#         dataset="classification")

# test_dl = DataLoader(dataset, batch_size=4, num_workers=2, drop_last=False)

# model = pointMLP(num_classes=2)
# model = model.to(dev)
# model = torch.nn.DataParallel(model)
# state_dict = torch.load("200epochs.pt")["state_dict"]
# model.load_state_dict(state_dict)

# eval_model_classification(model, test_dl)